In [1]:
import os
path = "/Users/parker.erickson/supreme_court_transcripts/oyez/cases/"
cases = os.listdir(path)

In [2]:
print(len(cases))

16425


In [3]:
import json

validCases = []

for case in cases:
    try:
        caseDetails = json.load(open(path + case))
        if type(caseDetails) == dict:
            if (caseDetails["facts_of_the_case"] is not None and caseDetails["facts_of_the_case"] != '') and (caseDetails["question"] is not None and caseDetails["question"] != '') and (caseDetails["decisions"] is not None and caseDetails["decisions"] != ''):
                validCases.append(caseDetails)
    except:
        pass

In [4]:
len(validCases)

3122

In [26]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  cleantext = cleantext.strip('\n')
  return cleantext

In [27]:
def parseCase(case):
    try:
        cn = case["name"].split("v.")
        plantiff = cn[0].strip()
        defendant = cn[1].strip()
        question = cleanhtml(case["question"])
        facts = cleanhtml(case["facts_of_the_case"])

        question = question.replace(plantiff, "plantiff").replace(defendant, "defendant")
        facts = facts.replace(plantiff, "plantiff").replace(defendant, "defendant")

        winningParty = case["decisions"][0]["winning_party"]
        inFavorPlantiff = True if winningParty == plantiff or winningParty == case["first_party"] else False
        court = case["heard_by"][0]["identifier"]
        votes = {}
        for vote in case["decisions"][0]["votes"]:
            if vote["vote"] != "none":
                if inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "plantiff"
                elif not inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif not inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "plantiff"
        return {
            "plantiff": plantiff,
            "defendant": defendant,
            "question": question,
            "facts": facts,
            "court": court,
            "winningParty": winningParty,
            "inFavorPlantiff": inFavorPlantiff,
            "votes": votes
        }
    except:
        pass

In [28]:
processedCases = []
for case in validCases:
    paresed = parseCase(case)
    if paresed is not None:
        processedCases.append(paresed)

In [29]:
len(processedCases)

2938

In [30]:
processedCases[0]

{'plantiff': 'Calderone',
 'defendant': 'Thompson',
 'question': "Did the Court of Appeal's order recalling its mandate denying Thomas M. defendant all habeas relief violate 28 USC section 2244(b) as amended by the Antiterrorism and Effective Death Penalty Act of 1996? Was the order an abuse of the appellate court's discretion?",
 'facts': "In 1983, Thomas M. defendant was convicted of the rape and murder of Ginger Fleischli in California state court. The special circumstance found by the jury of murder during the commission of rape made defendant eligible for the death penalty. In 1995, a federal District Court invalidated defendant's death sentence by granted relief on his rape conviction and the rape special circumstance. In reversing, the Court of Appeals reinstated defendant's death sentence, noting that the State presented strong evidence of rape at trial. The Court of Appeals then issued a mandate denying all habeas relief. Two days before defendant's execution, the Court of App

In [31]:
processedCases[3]

{'plantiff': 'Bucklew',
 'defendant': 'Precythe',
 'question': 'Does the Eighth Amendment require an inmate with a unique and severe medical condition to prove an adequate alternative method of execution when raising an as-applied challenge to the state-authorized method of execution?\nWhat evidence is required for a court to determine whether an inmate’s proposed alternative method of execution significantly reduces the risk of severe pain as compared to the state’s method?\nMay a court evaluating an as-applied challenge to a state’s method of execution assume that medical personnel on the execution team are competent to manage the inmate’s condition?\nDid the petitioner meet his burden in proposing an alternative execution method under Glossip v. Gross?',
 'facts': 'Russell plantiff was convicted by a state court jury of murder, kidnapping, and rape, and was sentenced to death. After exhausting the state appeals process, plantiff was scheduled to be executed on May 21, 2014. He then 

In [9]:
import pickle

with open('processedCases.pickle', 'wb') as handle:
    pickle.dump(processedCases, handle, protocol=pickle.HIGHEST_PROTOCOL)